In [8]:
#dataset 
# After this we will use this data to find the vocab size:
# read it in to inspect it
import torch
from torch import nn
from torch.nn import functional as F

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

# okay now get the vocabulary
vocab = set(text)

len(vocab)

# we would like vocab to be a sorted list
vocab = sorted(list(vocab))
print(''.join(vocab))

# Now we need to map a number to each token(letter) so as to input it to the model
# and also have the reverse mapping so that we can convert the number back to letter on receiving output
stoi = { ch : i for i, ch in enumerate(vocab)}
itos = { i : ch for i, ch in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i.item()] for i in l])  # Convert tensor to Python int

enc_text = encode(text)

data = torch.tensor(enc_text)

print(data.shape, data.dtype)

# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

block_size = 8
train_data[:block_size+1]

batch_size = 4

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')



for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")


n_embd = 32
dropout = 0.3

token_embedding_table = nn.Embedding(len(vocab), n_embd)


# self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time(or block), channels

x_out =token_embedding_table(xb)
# let's see a single Head perform self-attention
head_size = 32
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x_out)   # (B, T, 32)
q = query(x_out) # (B, T, 32)
wei =  q @ k.transpose(-2, -1) * head_size**-0.5# (B, T, 32) @ (B, 32, T) ---> (B, T, T) 
# Attention weights calculated

tril = torch.tril(torch.ones(T, T)) # lower traingular matrix
wei = wei.masked_fill(tril == 0, float('-inf')) # lower triangular matrix untouched as upper one stores neg.inf
wei = F.softmax(wei, dim=-1) #perform softmax along the rows

v = value(x_out)
out = wei @ v

print(out.shape)

ffn_c1 = nn.Linear(n_embd, 4 * n_embd)  # projection up
ffn_c2 = nn.Linear(4 * n_embd, n_embd)   # projection down
ln1 = nn.LayerNorm(n_embd)  # layernorm for attention
ln2 = nn.LayerNorm(n_embd)  # layernorm for ffn
drop = nn.Dropout(dropout)

x_out = x_out + out
x_out = ln1(x_out)

# feed forward network
x_out = ffn_c2(F.relu(ffn_c1(x_out)))
x_out = drop(x_out)  # apply dropout
x_out = ln2(x_out)   # final layer norm


# for sequence prediction, add final projection to vocabulary size
vocab_size = 65 
lm_head = nn.Linear(n_embd, vocab_size)
logits = lm_head(x_out)  # (B, T, vocab_size)

# get predictions
probs = F.softmax(logits, dim=-1)
# if you want just the next token prediction:
next_token_logits = logits[:, -1, :]  # (B, vocab_size)

print(f"Full sequence logits shape: {logits.shape}")  # [4, 8, 100]
print(f"Next token logits shape: {next_token_logits.shape}")  # [4, 100]



length of dataset in characters:  1115394

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
torch.Size([1115394]) torch.int64
inputs:
torch.Size([4, 8])
tensor([[58, 46, 39, 52,  1, 44, 43, 43],
        [ 1, 44, 53, 53, 58,  0, 20, 43],
        [19, 47, 60, 43,  1, 51, 43,  1],
        [53, 59,  1, 51, 59, 57, 58,  6]])
targets:
torch.Size([4, 8])
tensor([[46, 39, 52,  1, 44, 43, 43, 58],
        [44, 53, 53, 58,  0, 20, 43,  1],
        [47, 60, 43,  1, 51, 43,  1, 58],
        [59,  1, 51, 59, 57, 58,  6,  1]])
----
when input is [58] the target: 46
when input is [58, 46] the target: 39
when input is [58, 46, 39] the target: 52
when input is [58, 46, 39, 52] the target: 1
when input is [58, 46, 39, 52, 1] the target: 44
when input is [58, 46, 39, 52, 1, 44] the target: 43
when input is [58, 46, 39, 52, 1, 44, 43] the target: 43
when input is [58, 46, 39, 52, 1, 44, 43, 43] the target: 58
when input is [1] the target: 44
when input is [1, 44] the target: 53
when input 

In [13]:
import torch
from torch import nn
from torch.nn import functional as F

# Initialize hyperparameters
n_embd = 32
block_size = 8
batch_size = 4
vocab_size = len(vocab)
dropout = 0.3
learning_rate = 0.00003
num_iterations = 1000000
log_interval = 100

# Define the model components
token_embedding_table = nn.Embedding(vocab_size, n_embd)
key = nn.Linear(n_embd, n_embd, bias=False)
query = nn.Linear(n_embd, n_embd, bias=False)
value = nn.Linear(n_embd, n_embd, bias=False)
ffn_c1 = nn.Linear(n_embd, 4 * n_embd)
ffn_c2 = nn.Linear(4 * n_embd, n_embd)
ln1 = nn.LayerNorm(n_embd)
ln2 = nn.LayerNorm(n_embd)
drop = nn.Dropout(dropout)
lm_head = nn.Linear(n_embd, vocab_size)

# Combine components into a forward pass
def model_forward(x):
    x_emb = token_embedding_table(x)

    # Self-attention
    k = key(x_emb)
    q = query(x_emb)
    v = value(x_emb)

    wei = q @ k.transpose(-2, -1) * (n_embd**-0.5)
    tril = torch.tril(torch.ones(block_size, block_size, device=x.device))
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)

    out = wei @ v
    x = x_emb + out
    x = ln1(x)

    # Feed-forward network
    x = x + drop(ffn_c2(F.relu(ffn_c1(x))))
    x = ln2(x)

    # Final projection to vocab size
    logits = lm_head(x)
    return logits

# Define optimizer and loss criterion
optimizer = torch.optim.Adam(
    list(token_embedding_table.parameters()) +
    list(key.parameters()) +
    list(query.parameters()) +
    list(value.parameters()) +
    list(ffn_c1.parameters()) +
    list(ffn_c2.parameters()) +
    list(ln1.parameters()) +
    list(ln2.parameters()) +
    list(lm_head.parameters()),
    lr=learning_rate
)
criterion = nn.CrossEntropyLoss()

# Training loop
for iteration in range(num_iterations):
    # Get a batch of training data
    xb, yb = get_batch('train')

    # Forward pass
    logits = model_forward(xb)
    # Reshape logits and targets for loss computation
    B, T, C = logits.shape
    loss = criterion(logits.view(B * T, C), yb.view(B * T))

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Logging
    if iteration % log_interval == 0:
        with torch.no_grad():
            # Evaluate on validation set
            val_xb, val_yb = get_batch('val')
            val_logits = model_forward(val_xb)
            val_loss = criterion(val_logits.view(B * T, C), val_yb.view(B * T))
        
        print(f"Iteration {iteration}, Train Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

print("Training complete.")


Iteration 0, Train Loss: 4.3902, Validation Loss: 4.2025
Iteration 100, Train Loss: 4.3152, Validation Loss: 4.1620
Iteration 200, Train Loss: 4.1820, Validation Loss: 3.9703
Iteration 300, Train Loss: 3.9463, Validation Loss: 4.2240
Iteration 400, Train Loss: 4.0025, Validation Loss: 4.1944
Iteration 500, Train Loss: 4.0149, Validation Loss: 4.1657
Iteration 600, Train Loss: 3.8793, Validation Loss: 3.7647
Iteration 700, Train Loss: 3.7156, Validation Loss: 3.9885
Iteration 800, Train Loss: 3.8511, Validation Loss: 3.8184
Iteration 900, Train Loss: 3.7978, Validation Loss: 3.6808
Iteration 1000, Train Loss: 3.7492, Validation Loss: 3.8188
Iteration 1100, Train Loss: 3.4992, Validation Loss: 3.7903
Iteration 1200, Train Loss: 3.6853, Validation Loss: 3.7850
Iteration 1300, Train Loss: 3.7897, Validation Loss: 3.4083
Iteration 1400, Train Loss: 3.4041, Validation Loss: 3.7673
Iteration 1500, Train Loss: 3.2916, Validation Loss: 3.4662
Iteration 1600, Train Loss: 3.6403, Validation Loss:

KeyboardInterrupt: 

In [ ]:

for iteration in range(num_iterations):
    # Get a batch of training data
    xb, yb = get_batch('train')

    # Forward pass
    logits = model_forward(xb)
    # Reshape logits and targets for loss computation
    B, T, C = logits.shape
    loss = criterion(logits.view(B * T, C), yb.view(B * T))

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Logging
    if iteration % log_interval == 0:
        with torch.no_grad():
            # Evaluate on validation set
            val_xb, val_yb = get_batch('val')
            val_logits = model_forward(val_xb)
            val_loss = criterion(val_logits.view(B * T, C), val_yb.view(B * T))
        
        print(f"Iteration {iteration}, Train Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

print("Training complete.")
